## 📋 Step 1: Environment Setup

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

In [ ]:
# Install required packages
!pip install -q pyyaml tensorboard thop albumentations pycocotools

print("✅ Dependencies installed!")

## 💾 Step 2: Mount Google Drive (for checkpoints)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create project directory in Google Drive
!mkdir -p "/content/drive/MyDrive/HE_YOLOX"
!mkdir -p "/content/drive/MyDrive/HE_YOLOX/checkpoints"
!mkdir -p "/content/drive/MyDrive/HE_YOLOX/logs"
!mkdir -p "/content/drive/MyDrive/HE_YOLOX/results"

print("✅ Google Drive mounted successfully!")

## 📦 Step 3: Upload Project Files

**Option A: Upload the entire project folder as a ZIP**
- Zip your `implement` folder locally
- Upload it using the cell below

**Option B: Clone from GitHub (if you pushed the code)**

In [ ]:
# Option A: Upload ZIP file
from google.colab import files
import zipfile
import os

print("📤 Upload your project ZIP file...")
uploaded = files.upload()

# Extract the ZIP
for filename in uploaded.keys():
    print(f"Extracting {filename}...")
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('/content')
    print(f"✅ Extracted {filename}")

# Navigate to project directory
%cd /content/implement
!ls -la

In [ ]:
# Option B: Clone from GitHub (uncomment if using)
# !git clone https://github.com/YOUR_USERNAME/HE-YOLOX-ASFF.git
# %cd HE-YOLOX-ASFF

## 📊 Step 4: Download VisDrone2019 Dataset

In [ ]:
import os
import urllib.request
import zipfile
from tqdm.notebook import tqdm

# Create data directory
!mkdir -p data/VisDrone2019

# Dataset URLs
urls = {
    'train': 'https://github.com/VisDrone/VisDrone-Dataset/releases/download/v1.0/VisDrone2019-DET-train.zip',
    'val': 'https://github.com/VisDrone/VisDrone-Dataset/releases/download/v1.0/VisDrone2019-DET-val.zip',
    'test': 'https://github.com/VisDrone/VisDrone-Dataset/releases/download/v1.0/VisDrone2019-DET-test-dev.zip'
}

def download_and_extract(url, split_name):
    filename = f"data/VisDrone2019/{split_name}.zip"
    
    # Download
    print(f"\n📥 Downloading {split_name} set...")
    urllib.request.urlretrieve(url, filename)
    
    # Extract
    print(f"📦 Extracting {split_name} set...")
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('data/VisDrone2019')
    
    # Remove zip file
    os.remove(filename)
    print(f"✅ {split_name} set ready!")

# Download all splits
for split, url in urls.items():
    download_and_extract(url, split)

# Verify dataset
print("\n📊 Dataset Statistics:")
!echo "Training images: $(ls data/VisDrone2019/VisDrone2019-DET-train/images | wc -l)"
!echo "Validation images: $(ls data/VisDrone2019/VisDrone2019-DET-val/images | wc -l)"
!echo "Test images: $(ls data/VisDrone2019/VisDrone2019-DET-test-dev/images | wc -l)"

print("\n✅ Dataset downloaded and ready!")

## ⚙️ Step 5: Update Configuration for Colab

In [ ]:
import yaml

# Load config
with open('configs/he_yolox_asff.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Update paths for Colab
config['data']['data_dir'] = './data/VisDrone2019'
config['output']['model_dir'] = '/content/drive/MyDrive/HE_YOLOX/checkpoints'
config['output']['log_dir'] = '/content/drive/MyDrive/HE_YOLOX/logs'
config['output']['results_dir'] = '/content/drive/MyDrive/HE_YOLOX/results'

# Optimize for Colab (adjust batch size based on GPU memory)
config['train']['batch_size'] = 16  # T4 can handle 16
config['val']['batch_size'] = 16
config['train']['num_workers'] = 2  # Colab has limited CPU cores
config['val']['num_workers'] = 2

# Save updated config
with open('configs/he_yolox_asff_colab.yaml', 'w') as f:
    yaml.dump(config, f)

print("✅ Configuration updated for Colab!")
print(f"\nBatch size: {config['train']['batch_size']}")
print(f"Checkpoints will be saved to: {config['output']['model_dir']}")

## 🧪 Step 6: Test Model Architecture

In [ ]:
# Test model can be built
import torch
from models import build_he_yolox

print("Building HE-YOLOX-S model...")
model = build_he_yolox("s", num_classes=13)
model = model.cuda()

# Test forward pass
x = torch.randn(2, 3, 640, 640).cuda()
model.eval()
with torch.no_grad():
    outputs = model(x)

print(f"\n✅ Model test passed!")
print(f"Output shape: {outputs.shape}")

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal Parameters: {total_params / 1e6:.2f}M")
print(f"Trainable Parameters: {trainable_params / 1e6:.2f}M")

del model, x
torch.cuda.empty_cache()

## 🚀 Step 7: Start Training

**Important Notes:**
- Training will take 12-18 hours on T4 GPU
- Checkpoints are saved every 10 epochs to Google Drive
- If session times out, run this cell again with `--resume` flag
- Best model is automatically saved when validation loss improves

In [ ]:
# Check for existing checkpoint to resume
import os
import glob

checkpoint_dir = '/content/drive/MyDrive/HE_YOLOX/checkpoints'
checkpoints = sorted(glob.glob(f"{checkpoint_dir}/epoch_*.pth"))

if checkpoints:
    latest_checkpoint = checkpoints[-1]
    print(f"Found checkpoint: {latest_checkpoint}")
    print(f"Resume training? (y/n)")
    resume_flag = f"--resume {latest_checkpoint}"
else:
    print("No checkpoint found. Starting fresh training.")
    resume_flag = ""

# Start training
!python train.py \
    --config configs/he_yolox_asff_colab.yaml \
    --device cuda \
    --epochs 300 \
    {resume_flag}

## 📊 Step 8: Monitor Training (Optional)

Run this in a separate cell while training to monitor progress

In [ ]:
# Load TensorBoard
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/HE_YOLOX/logs

In [ ]:
# Monitor GPU usage
!nvidia-smi

## 📈 Step 9: Evaluate Model

In [ ]:
# Evaluate best model on validation set
!python eval.py \
    --config configs/he_yolox_asff_colab.yaml \
    --weights /content/drive/MyDrive/HE_YOLOX/checkpoints/best.pth \
    --split val \
    --device cuda

# Display results
print("\n" + "="*50)
print("Evaluation Results:")
print("="*50)
!cat /content/drive/MyDrive/HE_YOLOX/results/eval_results.txt

## 🎯 Step 10: Test Inference

In [ ]:
# Test on a few validation images
import glob
import random
import shutil
import os

# Create test directory
!mkdir -p test_images

# Copy 5 random validation images
val_images = glob.glob('data/VisDrone2019/VisDrone2019-DET-val/images/*.jpg')[:5]
for img in val_images:
    shutil.copy(img, 'test_images/')

print(f"Selected {len(val_images)} test images")

# Run inference
!python inference.py \
    --config configs/he_yolox_asff_colab.yaml \
    --weights /content/drive/MyDrive/HE_YOLOX/checkpoints/best.pth \
    --source test_images/ \
    --output /content/drive/MyDrive/HE_YOLOX/results/inference \
    --conf_thresh 0.3 \
    --save_img \
    --device cuda

print("\n✅ Inference complete! Check results in Google Drive.")

In [ ]:
# Display inference results
import matplotlib.pyplot as plt
import cv2
import glob
from PIL import Image

result_images = glob.glob('/content/drive/MyDrive/HE_YOLOX/results/inference/*.jpg')

# Display first 3 results
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
for idx, img_path in enumerate(result_images[:3]):
    img = Image.open(img_path)
    axes[idx].imshow(img)
    axes[idx].axis('off')
    axes[idx].set_title(f'Detection Result {idx+1}')

plt.tight_layout()
plt.show()

print(f"\nTotal results: {len(result_images)}")

## 💾 Step 11: Download Results

In [ ]:
# Create a ZIP of all results
import shutil
from google.colab import files

print("📦 Creating results archive...")
shutil.make_archive('he_yolox_results', 'zip', '/content/drive/MyDrive/HE_YOLOX')

print("\n📥 Download starting...")
files.download('he_yolox_results.zip')

print("\n✅ Results downloaded! Contains:")
print("  - Trained model weights (best.pth)")
print("  - All epoch checkpoints")
print("  - TensorBoard logs")
print("  - Evaluation results")
print("  - Inference visualizations")

## 📊 Training Summary

### Expected Results (from paper):

| Class | Target AP (%) |
|-------|---------------|
| Car | 81.2 |
| Bus | 66.4 |
| Truck | 55.6 |
| Pedestrian | 42.6 |
| Motor | 45.5 |
| Bicycle | 19.4 |

### Files in Google Drive:
- `/content/drive/MyDrive/HE_YOLOX/checkpoints/` - Model weights
- `/content/drive/MyDrive/HE_YOLOX/logs/` - Training logs
- `/content/drive/MyDrive/HE_YOLOX/results/` - Evaluation & inference results

### Tips:
1. **If session times out:** Just re-run the training cell - it will automatically resume
2. **Monitor progress:** Use TensorBoard or check checkpoint files
3. **Best model:** Automatically saved as `best.pth` when validation improves
4. **Full training:** Takes ~12-18 hours on T4 GPU

---

**🎉 Congratulations! You've successfully trained HE-YOLOX-ASFF!**

## 🚀 Next Steps

1. **Compare with paper results** - Check if your mAP matches the paper
2. **Fine-tune hyperparameters** - Adjust learning rate, batch size
3. **Test on custom images** - Upload your own drone images
4. **Export model** - Convert to ONNX for deployment
5. **Experiment** - Try different model sizes (M, L, X)

### Resources:
- Paper: DOI 10.1109/ICDSCNC62492.2024.10939462
- VisDrone Dataset: http://aiskyeye.com/
- Your results: `/content/drive/MyDrive/HE_YOLOX/`